In [3]:
import pandas as pd
import numpy as np
#import torch
import math
import random
#import sklearn as skl

In [4]:
data = pd.read_csv("triplets_all.csv")
data

,ids,intersize
0,0_0,46
1,0_1,5
2,0_2,2
3,0_3,14
4,0_4,10
...,...,...
385876,875_876,0
385877,875_877,0
385878,876_876,43
385879,876_877,3


In [9]:
sz = [0] * n
ist = [[0] * n for i in range(n)]
f = open("graph.txt", "w")
for i in range(N):
    s1, s2 = data.iloc[i, 0], data.iloc[i, 1]
    x1, x2 = map(int, s1.split('_'))
    sz = int(s2)
    if (x1 >=n or x2 >= n):
        break
    f.write(str(x1+1))
    f.write(" ")
    f.write(str(x2+1))
    f.write(" ")
    f.write(str(sz))
    f.write("\n")
    #ist[x1][x2] = sz
    #ist[x2][x1] = sz
f.close()

In [5]:
n, N = 878, 385881

In [6]:
sz = [0] * n
ist = [[0] * n for i in range(n)]
for i in range(N):
    s1, s2 = data.iloc[i, 0], data.iloc[i, 1]
    x1, x2 = map(int, s1.split('_'))
    sz = int(s2)
    if (x1 >=n or x2 >= n):
        break
    ist[x1][x2] = sz
    ist[x2][x1] = sz

In [7]:
###############################
J = [[0] * n for i in range(n)]
for i in range(n):
    for j in range(n):
        J[i][j] = ist[i][j]
    J[i] = np.array(J[i])
J = np.array(J)
J

array([[ 46,   5,   2, ...,   0,   0,   0],
       [  5, 121,   8, ...,   0,   0,   0],
       [  2,   8,  43, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,  40,   0,   0],
       [  0,   0,   0, ...,   0,  43,   3],
       [  0,   0,   0, ...,   0,   3,  72]])

In [664]:
res = [[] for _ in range(n)]
c = 0
def greedy_1():
    global c, J, res
    imn = -1
    for i in range(n):
        if (J[i][i] > 0 and (imn == -1 or J[imn][imn] < J[i][i])):
            imn = i
    if (imn == -1):
        return
    cond = []
    for k in range(n):
        cond.append([-J[imn][k], k])
    cond.sort()
    for k in range(n):
        cond[k][0] = -cond[k][0]
    dgroup = [imn]
    for k in range(n):
        ngr = cond[k][1]
        if (ngr == imn):
            continue
        ok = 1
        for gr in dgroup:
            if (J[gr][ngr] == 0):
                ok = 0
        #for gr in dgroup:
        #    if (J[ngr][gr] == 0): 
        #        ok1 = 0
        #        break
        #for gr in dgroup:
        #    K_sum = (J[gr] > 0).sum() - J[gr][gr] - 1
        #    if (2 * J[gr][gr] < K_sum):
        #        ok2 = 0
        if (ok):
            dgroup.append(ngr)
    for ii in dgroup:
        for jj in dgroup:
            J[ii][jj] -= 1
        res[ii].append(c)
    c += 1

In [676]:
def max_klika():
    N = n
    gr_cp = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if J[i][j] != 0:
                gr_cp[i][j] = 1
        gr_cp[i][i] = 0
    v = [0] * N
    for i in range(N):
        sum = 0
        for j in range(N):
            sum += gr_cp[i][j]
        v[i] = sum
    while True:
        cnt = 0
        ind = -1; mn = 1e9
        for i in range(N):
            if v[i] != 0:
                cnt += 1
            if v[i] < mn and v[i] != 0:
                mn = v[i]
                ind = i
        if ind == -1:
            break
        if mn < cnt - 1:
            for j in range(N):
                if gr_cp[ind][j] == 1:
                    gr_cp[ind][j] = 0
                    gr_cp[j][ind] = 0
                    v[ind] -= 1
                    v[j] -= 1
        else:
            break
    arr = []
    mn = 100000000
    for i in range(N):
        if v[i] != 0:
            arr.append(i)
    
    for i in arr:
        for j in arr:
            if gr_cp[i][j] != 0:
                mn = min(mn, gr_cp[i][j])
    return (arr, int(mn))

In [673]:
###############################
J = [[0] * n for i in range(n)]
for i in range(n):
    for j in range(n):
        J[i][j] = ist[i][j]
    J[i] = np.array(J[i])
J = np.array(J)
J
res = [[] for _ in range(n)]
c = 0

In [674]:
def greedy():
    global c, J, res
    cond = []
    dgroup, mn = max_klika()
    s = 0
    for i in dgroup:
        s += J[i][i]
        cond.append([-J[i][i], i])
    s /= len(dgroup)
#     for k in range(n):
#         ngr = cond[k][1]
#         ok = 1
#         for gr in dgroup:
#             if (J[ngr][gr] == 0): 
#                 ok = 0
#                 break
#         if (ok):
#             dgroup.append(ngr)
    dgroup = []
    mxx = J.max()
    for i in cond:
        i[0] = -i[0]
        if (i[0] >= 50):
            dgroup.append(i[1])
    if (len(dgroup) <= 2):
        print("bad", s)
        dgroup = []
        for i in cond:
            dgroup.append(i[1])
    mn = 1000
    for i in dgroup:
        for j in dgroup:
            if (i == j):
                continue
            mn = min(mn, J[i][j])
    print(mn)
    for ii in dgroup:
        for jj in dgroup:
            J[ii][jj] -= mn
        u = c
        for kk in range(int(mn)):
            res[ii].append(u)
            u += 1
    c += int(mn)

In [677]:
for i in range(500):
    greedy()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
1
1
1
1
1
1
1
1
1
2
1
4
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1


In [672]:
c

5588

for i in range(n):
    if (J[0][i] != 0):
        print(i, j)

In [622]:
J

array([[36,  3,  2, ...,  0,  0,  0],
       [ 3, 80,  6, ...,  0,  0,  0],
       [ 2,  6, 31, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 24,  0,  0],
       [ 0,  0,  0, ...,  0, 43,  3],
       [ 0,  0,  0, ...,  0,  3, 59]])

In [670]:
All = []
for i in range(n):
    All.append([J[i][i], i])
All.sort()
All

[[-3701, 163],
 [-2549, 192],
 [-2348, 52],
 [-2147, 49],
 [-1687, 84],
 [-1666, 61],
 [-1607, 120],
 [-1445, 35],
 [-1097, 71],
 [-920, 34],
 [-885, 95],
 [-874, 82],
 [-855, 105],
 [-834, 22],
 [-808, 289],
 [-792, 477],
 [-755, 5],
 [-737, 232],
 [-718, 166],
 [-717, 233],
 [-704, 265],
 [-673, 157],
 [-646, 76],
 [-644, 188],
 [-627, 13],
 [-622, 207],
 [-600, 99],
 [-595, 79],
 [-590, 348],
 [-580, 56],
 [-574, 343],
 [-565, 250],
 [-561, 239],
 [-533, 259],
 [-528, 23],
 [-528, 381],
 [-510, 264],
 [-509, 223],
 [-503, 40],
 [-492, 64],
 [-489, 236],
 [-488, 345],
 [-486, 208],
 [-483, 124],
 [-482, 50],
 [-478, 344],
 [-475, 155],
 [-470, 51],
 [-466, 158],
 [-450, 17],
 [-443, 129],
 [-434, 271],
 [-425, 342],
 [-420, 141],
 [-410, 3],
 [-405, 151],
 [-404, 31],
 [-402, 138],
 [-400, 178],
 [-396, 57],
 [-388, 261],
 [-385, 63],
 [-378, 386],
 [-376, 94],
 [-368, 389],
 [-366, 359],
 [-355, 457],
 [-354, 235],
 [-352, 435],
 [-351, 458],
 [-343, 371],
 [-336, 180],
 [-334, 193]

array([   23,    65,    28,    23,    46,    58,    50,    73,    69,
          41,    29,    13,    30,    10,    32,    18,    29,    51,
          39,    19,    42,    62,    14,    86,    19,    46,    22,
          58,    54,    29,    26,    39,    22,    54,     4,     7,
          63,    36,    18,    64,    95,    46,    62,    33,    48,
          53,    51,    57,    20,    16,    13,    52,     0,    10,
          15,    54,    17,     0,    63,    35,    25,     2,    18,
           0,     0,    31,    31,    14,    29,    40,    18,     0,
          37,    47,    24,    23,    80,    28,    37,    15,    34,
          40,    11,    16,     6,    22,     0,    21,     8,    30,
          16,    52,     4,    41,     0,    19,    38,    63,    38,
          83,    38,    36,    29,    49,    36,    28,    56,    31,
          35,    42,    36,    25,    43,    23,    41,    35,    27,
          18,    12,    11,     0,     8,    23,    58,    80,    30,
          42,    19,

In [47]:
res

[[9,
  14,
  62,
  69,
  89,
  134,
  169,
  236,
  238,
  267,
  270,
  289,
  347,
  381,
  436,
  437,
  467,
  502,
  586,
  632,
  647,
  661,
  667,
  711,
  842,
  879,
  880,
  901,
  982,
  1065,
  1081],
 [0,
  1,
  4,
  5,
  8,
  15,
  16,
  21,
  33,
  34,
  35,
  49,
  62,
  65,
  66,
  76,
  91,
  98,
  101,
  105,
  130,
  132,
  140,
  142,
  144,
  202,
  233,
  236,
  248,
  260,
  283,
  284,
  292,
  310,
  315,
  319,
  380,
  400,
  404,
  408,
  425,
  441,
  459,
  485,
  492,
  493,
  503,
  507,
  513,
  531,
  535,
  554,
  563,
  569,
  577,
  590,
  609,
  621,
  683,
  686,
  735,
  770,
  797,
  803,
  832,
  881,
  930,
  953,
  988,
  1017,
  1058,
  1101],
 [51,
  59,
  82,
  87,
  119,
  140,
  164,
  228,
  233,
  312,
  318,
  341,
  428,
  459,
  495,
  565,
  577,
  634,
  637,
  648,
  700,
  775,
  814,
  821,
  947,
  965,
  1044,
  1088],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  

In [361]:
sum_ = 0
for i in All:
    sum_ += -i[0]
sum_

24559

In [363]:
M = c
N = 878

In [364]:
matr = np.zeros((N, M))
for i in range(N):
    for j in range(len(res[i])):
        #print(res[i][j])
        matr[i][res[i][j]] = 1
        #matr[res[i][j]][i] = 1

In [346]:
matr

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [365]:
G = np.matmul(matr, matr.T)

In [382]:
G[0]

array([74.,  5.,  2.,  7.,  5.,  7.,  2.,  3.,  2.,  6.,  5.,  1.,  2.,
        5.,  1.,  1.,  3.,  3.,  2.,  3.,  4.,  3., 11.,  5.,  0.,  0.,
        2.,  2.,  2.,  1.,  1.,  5.,  3.,  4., 13., 15.,  4.,  4.,  0.,
        7.,  3.,  0.,  3.,  2.,  2.,  1.,  6.,  3.,  2., 20., 12.,  3.,
       23.,  6.,  2.,  4.,  5.,  4.,  2.,  4.,  2., 17.,  2.,  6.,  7.,
        2.,  3.,  1.,  5.,  2.,  0., 10.,  2.,  5.,  1.,  0.,  8.,  2.,
        0.,  7.,  0.,  1.,  7.,  1.,  8.,  0.,  2.,  1.,  8.,  2.,  1.,
        5.,  1.,  1.,  4.,  7.,  4.,  3.,  3.,  6.,  2.,  1.,  1.,  2.,
        4., 11.,  4.,  4.,  2.,  3.,  0.,  1.,  1.,  1.,  2.,  2.,  4.,
        0.,  1.,  1., 14.,  1.,  1.,  4.,  4.,  3.,  2.,  3.,  0.,  4.,
        3.,  2.,  0.,  6.,  4.,  1.,  0.,  3.,  8.,  2.,  1.,  7.,  3.,
        1.,  2.,  2.,  3.,  5.,  2.,  2.,  1.,  7.,  0.,  2.,  1.,  4.,
        1., 10.,  5.,  1.,  1.,  2.,  7., 16.,  1.,  1.,  6.,  1.,  2.,
        3.,  3.,  1.,  1.,  2.,  0.,  4.,  2.,  2.,  2.,  1.,  3

In [383]:
J[0]

array([46,  5,  2, 14, 10, 14,  9,  3,  7, 10,  6,  1,  3, 16,  1,  4,  3,
        9,  4,  3,  7,  7, 29, 13,  2,  5,  5,  3,  7,  6,  3,  5,  3,  5,
       21, 24, 12,  7,  2,  7,  6,  6,  5,  4,  4,  4, 10,  9,  4, 32, 15,
        9, 36, 11,  2,  5, 15,  4,  3,  6,  6, 29,  6,  8, 10,  2,  4,  1,
        6,  6,  2, 18,  2,  6,  2,  1, 10,  2,  2, 12,  2,  1, 11,  4, 15,
        3,  3,  3,  8,  2,  1,  9,  1,  2, 10, 17,  4,  5,  3, 11,  2,  1,
        3,  8,  4, 23,  4,  7,  2,  3,  3,  1,  1,  1,  2,  2,  5,  3,  2,
        3, 24,  1,  1,  5,  7,  3,  2,  3,  2, 12,  3,  3,  5,  6,  6,  3,
        1,  4,  8,  3,  3, 13,  3,  7,  2,  2,  4,  6,  2,  2,  1,  7,  1,
        2,  1,  8,  1, 13,  5,  1,  3,  6,  8, 23,  2,  3,  6,  5,  2,  7,
        3,  4,  2, 12,  3,  5,  3,  5,  4,  2,  4,  2,  3,  3,  2,  2,  3,
        1,  7,  1,  4,  1, 20,  3,  4,  3,  1,  3,  3,  1,  4,  1,  2,  3,
        3,  4,  1,  3, 11,  1,  3,  7,  4,  5,  3,  3,  1,  4,  2,  5,  1,
        1,  1,  3,  4,  1

In [381]:
for i in range(N):
    for j in range(M):
        if (G[i][j] != J[i][j]):
            print(i, j)

0 0
0 3
0 4
0 5
0 6
0 8
0 9
0 10
0 12
0 13
0 15
0 17
0 18
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 33
0 34
0 35
0 36
0 37
0 38
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 55
0 56
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 66
0 68
0 69
0 70
0 71
0 73
0 74
0 75
0 76
0 78
0 79
0 80
0 82
0 83
0 84
0 85
0 86
0 87
0 91
0 93
0 94
0 95
0 97
0 99
0 102
0 103
0 105
0 107
0 110
0 116
0 117
0 118
0 119
0 120
0 123
0 124
0 128
0 129
0 131
0 132
0 134
0 135
0 136
0 137
0 139
0 140
0 141
0 143
0 146
0 147
0 152
0 155
0 157
0 160
0 161
0 162
0 163
0 164
0 165
0 167
0 169
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184
0 185
0 186
0 188
0 190
0 192
0 193
0 194
0 198
0 200
0 202
0 203
0 204
0 205
0 206
0 208
0 211
0 212
0 213
0 215
0 217
0 219
0 220
0 224
0 230
0 231
0 232
0 233
0 234
0 236
0 239
0 240
0 243
0 244
0 245
0 247
0 248
0 249
0 250
0 253
0 254
0 255
0 256
0 257
0 258
0 259
0 260
0 261
0 262
0 263
0 264
0 265
0 267
0 27

IndexError: index 878 is out of bounds for axis 0 with size 878

In [370]:
np.abs(J - G).sum()

1241873.0

In [374]:
T = J - G

In [375]:
for i in range(N):
    for j in range(M):
        if (T[i][j]):
            print(T[i][j], i, j)

-28.0 0 0
7.0 0 3
5.0 0 4
7.0 0 5
7.0 0 6
5.0 0 8
4.0 0 9
1.0 0 10
1.0 0 12
11.0 0 13
3.0 0 15
6.0 0 17
2.0 0 18
3.0 0 20
4.0 0 21
18.0 0 22
8.0 0 23
2.0 0 24
5.0 0 25
3.0 0 26
1.0 0 27
5.0 0 28
5.0 0 29
2.0 0 30
1.0 0 33
8.0 0 34
9.0 0 35
8.0 0 36
3.0 0 37
2.0 0 38
3.0 0 40
6.0 0 41
2.0 0 42
2.0 0 43
2.0 0 44
3.0 0 45
4.0 0 46
6.0 0 47
2.0 0 48
12.0 0 49
3.0 0 50
6.0 0 51
13.0 0 52
5.0 0 53
1.0 0 55
10.0 0 56
1.0 0 58
2.0 0 59
4.0 0 60
12.0 0 61
4.0 0 62
2.0 0 63
3.0 0 64
1.0 0 66
1.0 0 68
4.0 0 69
2.0 0 70
8.0 0 71
1.0 0 73
1.0 0 74
1.0 0 75
2.0 0 76
2.0 0 78
5.0 0 79
2.0 0 80
4.0 0 82
3.0 0 83
7.0 0 84
3.0 0 85
1.0 0 86
2.0 0 87
4.0 0 91
1.0 0 93
6.0 0 94
10.0 0 95
2.0 0 97
5.0 0 99
2.0 0 102
6.0 0 103
12.0 0 105
3.0 0 107
3.0 0 110
1.0 0 116
3.0 0 117
1.0 0 118
2.0 0 119
10.0 0 120
1.0 0 123
3.0 0 124
2.0 0 128
8.0 0 129
1.0 0 131
5.0 0 132
2.0 0 134
2.0 0 135
1.0 0 136
1.0 0 137
1.0 0 139
2.0 0 140
6.0 0 141
6.0 0 143
1.0 0 146
1.0 0 147
1.0 0 152
4.0 0 155
3.0 0 157
2.0 0 160
4.0

IndexError: index 878 is out of bounds for axis 0 with size 878

In [5]:
pd = pd.read_csv("solution_clique.csv")

In [6]:
pd

,ids,intersize
0,0_1,5
1,0_2,2
2,0_3,14
3,0_4,10
4,0_5,14
...,...,...
384643,850_869,0
384644,850_870,4
384645,850_871,1
384646,850_872,2
